CMS 2019 general payments csv prep for transfer to SQL

In [ ]:
#import dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from datetime import date, datetime
import os, sys 
import io
import ee
import os


set options

In [ ]:
#  set_options to warn when we are working on a copy
pd.set_option('mode.chained_assignment','warn')

In [ ]:
# dataframe will display without being truncated
pd.set_option('display.max_rows', 150)
pd.set_option('display.max_columns', 100)
# pd.set_option('display.width', 768)

mount google drive

In [ ]:
#mounting the google drive to access the files
from google.colab import files
from google.colab import drive
drive.mount('/content/drive')

create dataframe

In [ ]:
gen_csv_path="/content/drive/My Drive/cms_19/general_payments.csv"

In [ ]:
gen_pay_df = pd.read_csv(gen_csv_path, index_col=None, dtype=str, header='infer', )


In [ ]:
#uncomment to get 30 row sample
# gen_pay_df.sample(30)

In [ ]:
#uncomment for list of columns
# gen_pay_df.columns.to_list()

Clean up general payments file



In [ ]:
#check content of covered recipient
covered = gen_pay_df['Covered_Recipient_Type'].value_counts()
covered

In [ ]:
#rename values in covered recipient column
gen_pay_df['Covered_Recipient_Type']=gen_pay_df.replace({'Covered Recipient Physician':'Physician', 'Covered Recipient Teaching Hospital':'Hospital'})


In [ ]:
# number of rows
len(gen_pay_df)

In [ ]:
# random selection of rows
gen_pay_df.sample()

In [ ]:
# how many null values in the dataframe
gen_pay_df.isnull().sum()

create offshoot table "travel" 

In [ ]:
# potentially interesting data related to travel reimbursements
# split off as separate table
travel_df=gen_pay_df[gen_pay_df['State_of_Travel'].notnull()]
travel_df.sample(5)

In [ ]:
# number of rows in table
len(travel_df)


In [ ]:
# drop null values 
travel_df.dropna(how='any', axis=1,inplace=True)


In [ ]:
# number of rows in table
len(travel_df)

In [ ]:
# random selection of data
travel_df.sample(3)

In [ ]:
# list of column names
travel_df.columns.to_list()

In [ ]:
# save as csv file
travel_df.to_csv('/content/drive/My Drive/cms_19/travel.csv', index=False,)

Return to general payments

In [ ]:
#number of rows
len(gen_pay_df)


In [ ]:
# drop change type column - only one value "New"
gen_pay_df.drop('Change_Type', axis=1, inplace=True)

In [ ]:
# verify deletion
if 'Change_Type' in gen_pay_df:
  print('true')
else:
  print('nope')

In [ ]:
# df.dropna(subset=['Value1','Value2','Value3'], how='all',inplace=True)
len(gen_pay_df)

In [ ]:
# lots of mostly empty columns
gen_pay_df.drop(['Teaching_Hospital_CCN', 'Teaching_Hospital_ID', 'Teaching_Hospital_Name', 
                          'Physician_Name_Suffix','Recipient_Province','Recipient_Postal_Code',
                          'Physician_License_State_code3', 'Physician_License_State_code4',
                          'Physician_License_State_code5', 'City_of_Travel', 'State_of_Travel', 
                          'Country_of_Travel', 'Name_of_Third_Party_Entity_Receiving_Payment_or_Transfer_of_Value',
                          'Third_Party_Equals_Covered_Recipient_Indicator', 'Contextual_Information', 
                          ], axis=1, inplace=True)

In [ ]:
# number of rows
len(gen_pay_df)

In [ ]:
# unique values by count 
gen_pay_df['Number_of_Payments_Included_in_Total_Amount'].value_counts()

In [ ]:
# unique values by count 
gen_pay_df['Third_Party_Payment_Recipient_Indicator'].value_counts()

In [ ]:
gen_pay_df.drop(['Physician_Middle_Name', 'Recipient_Primary_Business_Street_Address_Line2', 'Physician_License_State_code2', 
                          'Third_Party_Payment_Recipient_Indicator','Delay_in_Publication_Indicator','Covered_or_Noncovered_Indicator_2',
                          'Indicate_Drug_or_Biological_or_Device_or_Medical_Supply_2', 'Product_Category_or_Therapeutic_Area_2',
                          'Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_2', 'Associated_Drug_or_Biological_NDC_2', 'Covered_or_Noncovered_Indicator_3', 
                          'Indicate_Drug_or_Biological_or_Device_or_Medical_Supply_3', 'Product_Category_or_Therapeutic_Area_3',
                          'Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_3', 'Associated_Drug_or_Biological_NDC_3', 'Covered_or_Noncovered_Indicator_4',
                          'Indicate_Drug_or_Biological_or_Device_or_Medical_Supply_4', 'Product_Category_or_Therapeutic_Area_4', 'Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_4',
                          'Associated_Drug_or_Biological_NDC_4', 'Covered_or_Noncovered_Indicator_5', 'Indicate_Drug_or_Biological_or_Device_or_Medical_Supply_5', 
                          'Product_Category_or_Therapeutic_Area_5', 'Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_5','Associated_Drug_or_Biological_NDC_5'], axis=1, inplace=True)

In [ ]:
# random selection of data
gen_pay_df.sample(3)

In [ ]:
# list of column names
gen_pay_df.columns.to_list()

In [ ]:
# remove misc ' "
gen_pay_df['Recipient_Primary_Business_Street_Address_Line1']=gen_pay_df['Recipient_Primary_Business_Street_Address_Line1'].str.replace(r"[\"\',]", '')

In [ ]:
# split specialty column 
gen_pay_df['Phys_Specialty'] = gen_pay_df['Physician_Specialty'].str.split('|').str[1:3]

In [ ]:
# random selection of data
gen_pay_df.sample()

In [ ]:
# unique values by count 
gen_pay_df['Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name'].value_counts()

In [ ]:
# unique values by count 
gen_pay_df['Physician_Specialty'].value_counts()

In [ ]:
# unique values by count 
gen_pay_df['Physician_Specialty'].value_counts()


In [ ]:
# unique values by count 
vc = gen_pay_df['Physician_Specialty'].value_counts()


In [ ]:
#many categories, not sure what I want to do long term with this column
# vc[vc > 20000].index[0:20]
vc[vc > 2000] 

In [ ]:
#statistics
gen_pay_df.describe()

In [ ]:
# select a row
gen_pay_df.sample()

In [ ]:
# rename column 
gen_pay_df.rename(columns={'Recipient_Primary_Business_Street_Address_Line1': 'Recipient_Street_Address'}, inplace=True)

In [ ]:
# save to csv
gen_pay_df.to_csv('/content/drive/My Drive/cms_19/payments.csv', index=False, header=True, )